In [10]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
from PIL import Image

import data_load
from os import listdir
from os.path import isfile, join

from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import model

In [11]:
video = 'OldVideoClip_Trim.mp4'

def CreateImageFromVideo(video):
    cam = cv2.VideoCapture(video)
    images = []
  
    try:
        if not os.path.exists('data'):
            os.makedirs('data')
    except OSError:
        print ('Error: Creating directory of data')
    
    currentframe = 0
  
    while(True):
        ret, frame = cam.read()
  
        if ret:
            name = './data/frame' + str(currentframe) + '.jpg'
            images.append('frame' +str(currentframe) + '.jpg')
            print ('Creating...' + name)

            cv2.imwrite(name, frame)
  
            currentframe += 1
        else:
            break

    cam.release()
    cv2.destroyAllWindows()
    
    return images


images = CreateImageFromVideo(video)

Creating..../data/frame0.jpg
Creating..../data/frame1.jpg
Creating..../data/frame2.jpg
Creating..../data/frame3.jpg
Creating..../data/frame4.jpg
Creating..../data/frame5.jpg
Creating..../data/frame6.jpg
Creating..../data/frame7.jpg
Creating..../data/frame8.jpg
Creating..../data/frame9.jpg
Creating..../data/frame10.jpg
Creating..../data/frame11.jpg
Creating..../data/frame12.jpg
Creating..../data/frame13.jpg
Creating..../data/frame14.jpg
Creating..../data/frame15.jpg
Creating..../data/frame16.jpg
Creating..../data/frame17.jpg
Creating..../data/frame18.jpg
Creating..../data/frame19.jpg
Creating..../data/frame20.jpg
Creating..../data/frame21.jpg
Creating..../data/frame22.jpg
Creating..../data/frame23.jpg
Creating..../data/frame24.jpg
Creating..../data/frame25.jpg
Creating..../data/frame26.jpg
Creating..../data/frame27.jpg
Creating..../data/frame28.jpg
Creating..../data/frame29.jpg
Creating..../data/frame30.jpg
Creating..../data/frame31.jpg
Creating..../data/frame32.jpg
Creating..../data/fr

Creating..../data/frame281.jpg
Creating..../data/frame282.jpg
Creating..../data/frame283.jpg
Creating..../data/frame284.jpg
Creating..../data/frame285.jpg
Creating..../data/frame286.jpg
Creating..../data/frame287.jpg
Creating..../data/frame288.jpg
Creating..../data/frame289.jpg
Creating..../data/frame290.jpg
Creating..../data/frame291.jpg
Creating..../data/frame292.jpg
Creating..../data/frame293.jpg
Creating..../data/frame294.jpg
Creating..../data/frame295.jpg
Creating..../data/frame296.jpg
Creating..../data/frame297.jpg
Creating..../data/frame298.jpg
Creating..../data/frame299.jpg
Creating..../data/frame300.jpg
Creating..../data/frame301.jpg
Creating..../data/frame302.jpg
Creating..../data/frame303.jpg
Creating..../data/frame304.jpg
Creating..../data/frame305.jpg
Creating..../data/frame306.jpg
Creating..../data/frame307.jpg
Creating..../data/frame308.jpg
Creating..../data/frame309.jpg
Creating..../data/frame310.jpg
Creating..../data/frame311.jpg
Creating..../data/frame312.jpg
Creating

In [12]:
state_dict = torch.load('checkpoint3.pth')
print(state_dict.keys())

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'conv2.weight', 'bn2.weight', 'bn2.bias', 'bn2.running_mean', 'bn2.running_var', 'bn2.num_batches_tracked', 'conv3.weight', 'bn3.weight', 'bn3.bias', 'bn3.running_mean', 'bn3.running_var', 'bn3.num_batches_tracked', 'conv4.weight', 'bn4.weight', 'bn4.bias', 'bn4.running_mean', 'bn4.running_var', 'bn4.num_batches_tracked', 'conv5.weight', 'bn5.weight', 'bn5.bias', 'bn5.running_mean', 'bn5.running_var', 'bn5.num_batches_tracked', 'deconv6.weight', 'bn6.weight', 'bn6.bias', 'bn6.running_mean', 'bn6.running_var', 'bn6.num_batches_tracked', 'deconv7.weight', 'bn7.weight', 'bn7.bias', 'bn7.running_mean', 'bn7.running_var', 'bn7.num_batches_tracked', 'deconv8.weight', 'bn8.weight', 'bn8.bias', 'bn8.running_mean', 'bn8.running_var', 'bn8.num_batches_tracked', 'deconv9.weight', 'bn9.weight', 'bn9.bias', 'bn9.running_mean', 'bn9.running_var', 'bn9.num_batches_tracked', 'deconv1

In [13]:
gen = model.Generator()
gen.load_state_dict(state_dict)

<All keys matched successfully>

In [14]:
#gen = model.Generator()
#gen.load_state_dict(torch.load(os.path.join('E:\\DeepLearning with pytorch\\PyTorch\Colorization\\', 'gen-%d.pkl' % 0)))



In [15]:
def graphSomeSample(imagesOutputResult):
    fig, axs = plt.subplots(3, 4)
    for i, ax in enumerate(axs.flatten()):
        if i < len(images):
            ax.imshow(imagesOutputResult[i])
        else:
            ax.remove()
    plt.show()

In [16]:
#graphSomeSample(images)

In [17]:
def generate_video(images):
    print(len(images))
    imagesOutput = []
    imagesOutputResult = []
    
    
    image_folder = '.' # make sure to use your folder
    video_name = 'mygeneratedvideo3.avi'
    os.chdir("E:\\DeepLearning with pytorch\\PyTorch\\Colorization")
    
    fourcc = cv2.VideoWriter_fourcc('I','4','2','0')
    
    video = cv2.VideoWriter(video_name, fourcc, float(32.6), (640, 480))
    
    path = 'E:\\DeepLearning with pytorch\\PyTorch\\Colorization\\data\\'
    train_paths = [path+f for f in listdir(path) if isfile(join(path, f))]

    data_transform = transforms.Compose([data_load.Normalize(), data_load.ToTensor()])
    Color_Dataset_train = data_load.ColorizationDataset(path_list = train_paths, transform=data_transform)


    batch_size = 65

    test_loader = DataLoader(Color_Dataset_train, 
                              batch_size=batch_size,
                              shuffle=False, 
                              num_workers=0)

    print('Number of images: ', len(Color_Dataset_train))
    currentframe = 0
    
    
    for batch_i, data in enumerate(test_loader):
        #print('current batch: ' + str(batch_i))
        L = data['L']
        ab = data['ab']
        img = data['image']
        img = img.type(torch.FloatTensor)
        L = L.type(torch.FloatTensor)
        fake = gen(L)
        for i in range(len(fake)):
            imagesOutputResult.append(fake[i].detach().numpy().transpose((2, 1, 0)))
                                
    for image in imagesOutputResult:
        
        name = 'E:\\DeepLearning with pytorch\\PyTorch\\Colorization\\result\\' + str(currentframe) + '.jpg'
        print ('Creating...' + name)
        currentframe += 1
        norm_image = cv2.normalize(image, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

        norm_image = norm_image.astype(np.uint8)
        im = Image.fromarray(norm_image).convert('RGB')
        im.save(name)
        
        video.write(cv2.imread(name))
    cv2.destroyAllWindows()
    video.release()
    print('Finished check your folder')
    
    
    


In [18]:
generate_video(images)

326
Number of images:  326
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\0.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\1.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\2.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\3.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\4.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\5.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\6.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\7.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\8.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\9.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\10.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\11.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\12.jpg
Creating

Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\111.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\112.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\113.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\114.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\115.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\116.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\117.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\118.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\119.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\120.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\121.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\122.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\123.jpg
Creating...E

Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\220.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\221.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\222.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\223.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\224.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\225.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\226.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\227.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\228.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\229.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\230.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\231.jpg
Creating...E:\DeepLearning with pytorch\PyTorch\Colorization\result\232.jpg
Creating...E

In [8]:
#print(torch.cuda.get_arch_list())

['sm_37', 'sm_50', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'compute_37']
